### SQL Databases


In [12]:
import sqlite3
import os
os.makedirs('data/databases', exist_ok=True)

In [13]:
## create sample database
conn = sqlite3.connect('data/databases/company.db')
cursor = conn.cursor()

In [14]:
# create table
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
               (id INTEGER PRIMARY KEY,
                name TEXT, role TEXT,department TEXT, salary REAL)''')

In [15]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
               (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [16]:
# Insert sample data
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]


In [17]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)',employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)',projects)


In [18]:
cursor.execute('SELECT * FROM employees')
print('Employees: ' )
for row in cursor.fetchall():
    print(row)

Employees: 
(1, 'John Doe', 'Senior Developer', 'Engineering', 95000.0)
(2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000.0)
(3, 'Mike Johnson', 'Product Manager', 'Product', 110000.0)
(4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000.0)


In [19]:
conn.commit()
conn.close()

### Database Content Extraction


In [20]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [25]:
## Method1 SQLDatabase Utilitiy

db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

##get database info
print(db.get_table_info())
print('------------')
print(db.get_usable_table_names())
print('------------')
print(db._all_tables)


CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/
------------
['employees', 'projects']
------------
{'projects', 'employees'}


In [26]:
from typing import List
from langchain_core.documents import Document

## Method 2: Custom SQL to Document conversion
print("\n custom SQL Processing \n")

def sql_to_documents(db_path:str) -> List[Document]:
    """Convert SQL Dtabase to documents with context"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents=[]

    #Strategy: Create documents for each table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'; ")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]

        #get table schema
        cursor.execute(f'PRAGMA table_info({table_name})')
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]

        #get table data
        cursor.execute(f'SELECT * FROM {table_name}')
        rows = cursor.fetchall()

                # Create table overview document
        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"
        
        # Add sample records
        table_content += "Sample Records:\n"
        for row in rows[:5]:  # First 5 records
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"
            print(f"Added record to document: {record}")
        
        doc = Document(
            page_content=table_content,
            metadata={
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'
            }
        )       
        documents.append(doc)

        
     # Strategy 2: Create relationship documents
    # Example: Join employees and projects
    cursor.execute("""
        SELECT e.name, e.role, p.name as project_name, p.status
        FROM employees e
        JOIN projects p ON e.id = p.lead_id
    """)
    
    relationships = cursor.fetchall()
    rel_content = "Employee-Project Relationships:\n\n"
    for rel in relationships:
        rel_content += f"{rel[0]} ({rel[1]}) leads {rel[2]} - Status: {rel[3]}\n"
    
    rel_doc = Document(
        page_content=rel_content,
        metadata={
            'source': db_path,
            'data_type': 'sql_relationships',
            'query': 'employee_project_join'
        }
    )
    documents.append(rel_doc)
    
    conn.close()
    return documents



 custom SQL Processing 



In [27]:
sql_to_documents("data/databases/company.db")

Added record to document: {'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}
Added record to document: {'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}
Added record to document: {'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}
Added record to document: {'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}
Added record to document: {'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.0, 'lead_id': 1}
Added record to document: {'id': 2, 'name': 'Data Pipeline', 'status': 'Completed', 'budget': 80000.0, 'lead_id': 2}
Added record to document: {'id': 3, 'name': 'Customer Portal', 'status': 'Planning', 'budget': 200000.0, 'lead_id': 3}
Added record to document: {'id': 4, 'name': 'ML Platform', 'status': 'Active', 'budget': 250000.0, 'lead_id': 2}

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.